In [1]:
!pip install wikipedia-api
!pip install beautifulsoup4
!pip install spacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-23 18:04:06.811200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 18:04:08.368604: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No su

In [15]:
import wikipediaapi
import requests
from bs4 import BeautifulSoup
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')

In [16]:
def get_wikipedia_info(monument_name):
    wiki = wikipediaapi.Wikipedia('en')
    page = wiki.page(monument_name)
    if page.exists():
        page_html = requests.get(page.fullurl).text
        soup = BeautifulSoup(page_html, 'html.parser')
        main_text = ''
        for p in soup.select('div.mw-parser-output > p'):
            if 'mw-empty-elt' in p.get('class', []):
                continue
            main_text += p.get_text()
        doc = nlp(main_text)
        entities = []
        for ent in doc.ents:
            if ent.label_ in ['DATE', 'PERSON', 'ORG', 'GPE']:
                entities.append(ent.text)
        sentences = []
        for sent in doc.sents:
            sent_entities = [ent for ent in entities if ent in sent.text]
            if sent_entities:
                sentences.append(sent.text.strip())

        return pd.DataFrame(sentences[:10])

    else:
        return None


In [19]:
monument_name = 'Taj Mahal'
info_df = get_wikipedia_info(monument_name)
print(info_df)

                                                   0
0     The Taj Mahal (/ˌtɑːdʒ məˈhɑːl, ˌtɑːʒ-/;  lit.
1  'Crown of the Palace')[4][5][6] is an Islamic ...
2  It was commissioned in 1631 by the fifth Mugha...
3  Construction of the mausoleum was essentially ...
4  The Taj Mahal complex is believed to have been...
5  The construction project employed some 20,000 ...
6  Various types of symbolism have been employed ...
7  The Taj Mahal was designated as a UNESCO World...
8  It is regarded by many as the best example of ...
9  The Taj Mahal attracts more than 6 million vis...
